In [1]:
import os, re, json , glob

In [7]:
# import re
# from bs4 import BeautifulSoup

# # Sample README content (you can load this from a file instead)
# # readme_content = '''
# # <details>
# # <summary>Click the arrow to view options.</summary>

# #   - m: metre (unit for length).
# #   - Pa: pascal (unit for pressure).
# #   - K: kelvin jslj.

# # </details>
# # '''

def extract_bullets_with_brackets(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    results = {}

    bullet_pattern = re.compile(r"\s*-\s*(\w+):\s*([^\(]+?)(?:\s*\((.*?)\))?\.")

    for details in soup.find_all("details"):
        lines = details.get_text().splitlines()
        for line in lines:
            match = bullet_pattern.match(line)
            if match:
                symbol, description, bracket_info = match.groups()
                results[symbol] = {
                    "text1": description.strip(),
                    "text2": bracket_info.strip() if bracket_info else None
                }

    return results

# # # Run the function
# # bullets = extract_bullets_with_brackets(readme_content)

# # # Print neatly
# # from pprint import pprint
# # pprint(bullets)


In [8]:
rme = glob.glob('*/*README*')
rme

['native-horizontal-grid-type/README.md',
 'native-horizontal-grid-region/README.md',
 'model-calendar/README.md',
 'native-horizontal-grid-nominal-resolution/README.md',
 'model-component/README.md',
 'native-horizontal-grid-temporal-refinement/README.md',
 'native-horizontal-grid-grid-mapping/README.md',
 'native-horizontal-grid-truncation-method/README.md',
 'model-component-type/README.md',
 'native-vertical-grid-coordinate/README.md',
 'native-horizontal-grid-arrangement/README.md',
 'model/README.md',
 'reference/README.md',
 'native-vertical-grid-units/README.md',
 'grid/README.md']

In [9]:
# pm = list(DATA_DESCRIPTOR_CLASS_MAPPING.values())[-4]


def bullet_pydantic(pm):

    keys = ""

    for key, value in pm.__pydantic_fields__.items():
        rq = '✓' if value.is_required() else '✗'
        
        typename = getattr(value.annotation, '__name__', str(value.annotation))
        description = value.description or 'No description in model.'

        # print(f"- **`{key}`**  [**{typename}**]  {rq} : \n  {description}\n")
        # print(f"- `{key}` [{typename}] {rq} : {description}")
        # print(f"- `{key}`\n      Type       : [{typename}]\n      Required   : {rq}\n      Description: {description}\n")
        
        print(f"- **`{key}`**  \n  {rq} (**{typename}**)\n  {description.rstrip()}\n  ")
        
        keys += f"- **`{key}`**  \n  {rq} (**{typename}**)\n  {description.rstrip()}\n  "
        
    return keys

def bullet_names(keynames):
    keys = ""
    for key in keys:
        # print(f"- `{key}`")
        print(f"- **`{key}`**")
        keys += f"- **`{key}`**  \n  ? (**NoType**)\n  No Linked Pydantic Model \n  "
    return keys

In [ ]:
# for rd in rme:
#     path = os.path.dirname(rd.replace('README.md',''))
#     print(path)
#     mytype = path.split('/')[-1]
    
#     context = {
#     "@context": [
#         "../_context_",
#         {
#             "@base": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/",
#             "@vocab": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/"
#         }
#     ],
#     "@embed": "@always",
#     "@type": [
#         "universal",
#         f"wcrp:{mytype}",
#     ]
#     }
#     with open(f'./{path}/_context_', 'w') as f:
#         json.dump(context, f, indent=4)
    
#     branches = extract_bullets_with_brackets(open(rd).read())
#     # print(branches)
    
#     for k, v in branches.items():
#         validation = k.replace('_','-').replace(' ','-')
#         filename = validation.lower()
        
        
#         ld = {
#             "id": filename,
#             "type": [f"wcrp:{mytype}","universal"],
#             "validation-key": validation,

            
#         } 
        
#         if v['text2'] is not None:
            
#             ld['description'] =  v['text2']
#             ld['ui-label'] = v['text1']
            
#         else:
#             ld['description'] = v['text1']
#             ld['ui-label'] = validation.replace('-',' ').capitalize()
            
#         # print(ld)
        
#         # with open(f'./{path}/{filename}.json', 'w') as f:
#         #     json.dump(ld, f, indent=4)

native-horizontal-grid-type


NameError: name 'BeautifulSoup' is not defined

In [12]:
folders = glob.glob('*/')
folders

['native-horizontal-grid-type/',
 'source-type/',
 'native-horizontal-grid-region/',
 'model-calendar/',
 'license/',
 'native-horizontal-grid-nominal-resolution/',
 'archive/',
 'frequency/',
 'activity/',
 'model-component/',
 'native-horizontal-grid-temporal-refinement/',
 'legacy/',
 'native-horizontal-grid-grid-mapping/',
 'product/',
 'native-horizontal-grid-truncation-method/',
 'mip/',
 'grid-label/',
 'model-component-type/',
 'resolution/',
 'native-vertical-grid-coordinate/',
 'native-horizontal-grid-arrangement/',
 'model/',
 'organisation/',
 'reference/',
 'realm/',
 'native-vertical-grid-units/',
 'grid/',
 'model-family/']

In [23]:
import numpy as np
import esgvoc
from esgvoc.api.data_descriptors import DATA_DESCRIPTOR_CLASS_MAPPING

from cmipld.utils.git import get_path_url,get_repo_url,get_relative_path,url2io
from cmipld import prefix_url,resolve_prefix

missing_pydantic = []

for dir in folders:
    
    print(dir)
    
    # if content is empty, then delete folder. 
    
    try:
        keys = os.popen(f"jq -r 'recurse(.[]? // empty) | objects | keys_unsorted[]' {dir}*.json | sort | uniq -c | sort -nr").read()
        keynames = [i.split(' ')[-1] for i in keys.split('\n') if i.strip() != '']
        
        keynumbers = [int(i.split(' ')[-2]) for i in keys.split('\n') if i.strip() != '']
        avg = int(np.median(keynumbers))
        # print(keynames,keynumbers)
        
        
        
        different = [keynames[i] for i, x in enumerate(keynumbers) if x != avg]
        
        if different: print('The following keys are not present in all files:',different)


    except ValueError as e:
        # print(f"Error processing {dir}: {e}")
        continue

    
    dname = dir.strip('/').replace('-','_')
    pydantic = False
    
    match True:
        case _ if dname in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dname  
        case _ if dname + '_new' in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dname + '_new' 
        case _ if dir.strip('/') in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dir.strip('/')
        case _:
            missing_pydantic.append(dname)  # none matched
            
            print(f"------ \n Adding {dname} to DATA_DESCRIPTOR_CLASS_MAPPING")
    
    
    
    if pydantic:
        bullets = bullet_pydantic(DATA_DESCRIPTOR_CLASS_MAPPING[pydantic])
    else:
        bulltes = bullet_names(keynames)
    

    ldurl= ''
    content = get_path_url(dir).replace('wolfiex','wcrp-cmip')
    repo = get_repo_url().replace('wolfiex','wcrp-cmip')
    relpath = get_relative_path(dir)
    
    
    
    io = relpath.replace('src-data/',url2io(repo,'main',relpath))
    short = prefix_url(io)
    
    print(content,repo)
    print(relpath,short,io),
    break
    
    
    info = f'''
    
    <section id="info">
    
    # {dname.replace('-',' ').capitalize()}
    
    | Attribute | Details |
    | --- | --- |
    | JSON-LD | `{short}/<id>` |
    | Content | `[{io}]({io}]/<id>` |
    | Github | [{repo}](repo) |
    | --- | --- |
    | Type | `wrcp:{dname}` |
    | Pydantic class | `{pydantic}`: {DATA_DESCRIPTOR_CLASS_MAPPING[pydantic].__name__ if pydantic else ' Not yet implemented'} |
   

    </section>
    '''
    
    print(info)
    
    
    
    
    usage = '''
    <section id="usage">
    ## Usage
    
    
    ### Online Viewer 
    
    You can type the Content link followed by our <file ID> and then `.json` as the extension  to load the file in a browser. 
    
    E.g. 
    
    ### Getting a File. 



    ### Framing
    ```js
    frame = {
                "@context": %(url)s/_context_
                "@type": "wcrp:%(dname)s",}",    
                "keys we want": "",
                "@explicit": True

            }
            
    
    print(usage)

    ```

    ```python
    
    import cmipld
    
    cmipld.frame( prefix:path)

    ```
    </section>

    '''%{'url': url, 'dname': dname}

    print( usage )

native-horizontal-grid-type/
------ 
 Adding native_horizontal_grid_type to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
https://github.com/wcrp-cmip/WCRP-universe/tree/main/src-data/native-horizontal-grid-type https://github.com/wcrp-cmip/WCRP-universe
src-data/native-horizontal-grid-type universal:native-horizontal-grid-type https://wcrp-cmip.github.io/WCRP-universe/native-horizontal-grid-type


In [ ]:
DATA_DESCRIPTOR_CLASS_MAPPING

In [ ]:
keys